In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Grunddatein/Zwischendatein/CleanedDataCompleteNoNulls.csv')
distinct_values_verkäufer = df['sellerName'].unique()
print(f"Es gibt insgesamt {len(distinct_values_verkäufer)} verschiedene Verkäufer")

df_algorithmic_seller_names = pd.read_csv('../Grunddatein/Zwischendatein/PA-Adopters.csv')
df_algorithmic_seller_names = df_algorithmic_seller_names.rename(columns={'AlgorithmicSellerNames': 'sellerName'})  # rename column

df_non_algorithmic_seller_names = set(distinct_values_verkäufer) - set(df_algorithmic_seller_names['sellerName'].values)
df_non_algorithmic_seller_names = pd.DataFrame({'sellerName': list(df_non_algorithmic_seller_names)})


print(f"Es gibt insgesamt {len(df_non_algorithmic_seller_names)} Non-Algo Verkäufer")
print(f"Es gibt insgesamt {len(df_algorithmic_seller_names) } Algo Verkäufer")

#Drop Duplicates if necessary. Shoudnt be the case
df_non_algorithmic_seller_names = df_non_algorithmic_seller_names.drop_duplicates()
df_algorithmic_seller_names = df_algorithmic_seller_names.drop_duplicates()

print(f"Es gibt insgesamt {len(df_non_algorithmic_seller_names)} Non-Algo Verkäufer")
print(f"Es gibt insgesamt {len(df_algorithmic_seller_names) } Algo Verkäufer")

C:\Users\Robert Betschinger\AppData\Local\Temp\ipykernel_8092\1409483830.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Grunddatein/Zwischendatein/CleanedDataCompleteNoNulls.csv')


Es gibt insgesamt 2377 verschiedene Verkäufer
Es gibt insgesamt 1766 Non-Algo Verkäufer
Es gibt insgesamt 611 Algo Verkäufer
Es gibt insgesamt 1766 Non-Algo Verkäufer
Es gibt insgesamt 611 Algo Verkäufer


In [3]:

df_non_algorithmic_seller_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1766 entries, 0 to 1765
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sellerName  1766 non-null   object
dtypes: object(1)
memory usage: 27.6+ KB


In [4]:
df_algorithmic_seller_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611 entries, 0 to 610
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sellerName  611 non-null    object
dtypes: object(1)
memory usage: 9.5+ KB


In [5]:
seller_names = ['amazon', 'Amazon US', 'Amazon UK','Amazon Warehouse ']

# Drop rows with seller names in the seller_names list
df_algorithmic_seller_names = df_algorithmic_seller_names[~df_algorithmic_seller_names['sellerName'].isin(seller_names)]
df_non_algorithmic_seller_names = df_non_algorithmic_seller_names[~df_non_algorithmic_seller_names['sellerName'].isin(seller_names)]

# Transform the seller_names list into a DataFrame
amazon_seller_names_df = pd.DataFrame(seller_names, columns=['sellerName'])

print(f"Es gibt insgesamt {len(df_non_algorithmic_seller_names)} Non-Algo Verkäufer")
print(f"Es gibt insgesamt {len(df_algorithmic_seller_names) } Algo Verkäufer")
print(f"Es gibt insgesamt {len(amazon_seller_names_df) } Amazon Verkäufer")

Es gibt insgesamt 1766 Non-Algo Verkäufer
Es gibt insgesamt 607 Algo Verkäufer
Es gibt insgesamt 4 Amazon Verkäufer


In [6]:
amazon_asin_values = df.loc[df['sellerName'].isin(['amazon', 'Amazon Warehouse ', 'Amazon US', 'Amazon UK']), 'asin'].unique()
print(f"An so vielen Asins ist Amazon als Verkäufer eingetragen:{len(amazon_asin_values)}")

# Assuming the DataFrame is named 'df'
all_asin_values = df['asin'].unique()

# Subtract the values that exist in the 'amazon_asin_values' list
non_amazon_asin_values = set(all_asin_values) - set(amazon_asin_values)
print(f"An so vielen Asins ist Amazon nicht als Verkäufer eingetragen:{len(non_amazon_asin_values)}")
print(f"So Viele unterschiedliche Asins gibt es maximal {len(all_asin_values)}")

# Create a DataFrame with only Amazon-sold products
amazon_df = df[df['asin'].isin(amazon_asin_values)]
# Create a DataFrame with only non-Amazon-sold products
non_amazon_df = df[df['asin'].isin(non_amazon_asin_values)]

An so vielen Asins ist Amazon als Verkäufer eingetragen:743
An so vielen Asins ist Amazon nicht als Verkäufer eingetragen:251
So Viele unterschiedliche Asins gibt es maximal 994


In [6]:
non_amazon_check_number_asins = non_amazon_df['asin'].nunique()
print("Gegencheck", non_amazon_check_number_asins)
amazon_check = amazon_df['asin'].nunique()
print("Gegencheck", amazon_check)

Gegencheck 272
Gegencheck 716


In [7]:
## Amazon present products

In [ ]:
import pandas as pd

# Step 1: Group non_amazon_df by 'asin' and 'time'
amazon_df = amazon_df.groupby(['asin', 'time'])
#amazon_df = amazon_df.groupby(['asin'])
# Step 2: Define a function to filter groups with at least one algorithmic and one non-algorithmic seller
def filter_mixed_sellers(group):
    algorithmic_present = group['sellerName'].isin(df_algorithmic_seller_names['sellerName']).any()
    non_algorithmic_present = group['sellerName'].isin(df_non_algorithmic_seller_names['sellerName']).any()
    return algorithmic_present and non_algorithmic_present

# Step 3: Apply the function to the grouped data and save the results in a new DataFrame
mixed_sellers_df_amazon_products = amazon_df.filter(filter_mixed_sellers)

# Add a new column 'is_algorithmic' to the mixed_sellers_df DataFrame
mixed_sellers_df_amazon_products['is_algorithmic'] = mixed_sellers_df_amazon_products['sellerName'].isin(df_algorithmic_seller_names['sellerName'])

# Step 4: Investigate the new DataFrame to see who is winning the Buy Box more often
algorithmic_buybox_wins = mixed_sellers_df_amazon_products[mixed_sellers_df_amazon_products['sellerName'].isin(df_algorithmic_seller_names['sellerName']) & (mixed_sellers_df_amazon_products['buyBoxWinner'] == True)].shape[0]
non_algorithmic_buybox_wins = mixed_sellers_df_amazon_products[mixed_sellers_df_amazon_products['sellerName'].isin(df_non_algorithmic_seller_names['sellerName']) & (mixed_sellers_df_amazon_products['buyBoxWinner'] == True)].shape[0]
amazon_wins_buy_box = mixed_sellers_df_amazon_products[mixed_sellers_df_amazon_products['sellerName'].isin(amazon_seller_names_df['sellerName']) & (mixed_sellers_df_amazon_products['buyBoxWinner'] == True)].shape[0]

print(f"Algorithmic sellers won the Buy Box {algorithmic_buybox_wins} times.")
print(f"Non-algorithmic sellers won the Buy Box {non_algorithmic_buybox_wins} times.")
print(f"Amazon sellers won the Buy Box {amazon_wins_buy_box} times.")

In [9]:
## Non Amazon Products

In [ ]:
import pandas as pd

# Step 1: Group non_amazon_df by 'asin' and 'time'
grouped_non_amazon_df = non_amazon_df.groupby(['asin', 'time'])

# Step 2: Define a function to filter groups with at least one algorithmic and one non-algorithmic seller
def filter_mixed_sellers(group):
    algorithmic_present = group['sellerName'].isin(df_algorithmic_seller_names['sellerName']).any()
    non_algorithmic_present = group['sellerName'].isin(df_non_algorithmic_seller_names['sellerName']).any()
    return algorithmic_present and non_algorithmic_present

# Step 3: Apply the function to the grouped data and save the results in a new DataFrame
mixed_sellers_df = grouped_non_amazon_df.filter(filter_mixed_sellers)

# Add a new column 'is_algorithmic' to the mixed_sellers_df DataFrame
mixed_sellers_df['is_algorithmic'] = mixed_sellers_df['sellerName'].isin(df_algorithmic_seller_names['sellerName'])

# Step 4: Investigate the new DataFrame to see who is winning the Buy Box more often
algorithmic_buybox_wins = mixed_sellers_df[mixed_sellers_df['sellerName'].isin(df_algorithmic_seller_names['sellerName']) & (mixed_sellers_df['buyBoxWinner'] == True)].shape[0]
non_algorithmic_buybox_wins = mixed_sellers_df[mixed_sellers_df['sellerName'].isin(df_non_algorithmic_seller_names['sellerName']) & (mixed_sellers_df['buyBoxWinner'] == True)].shape[0]

print(f"Algorithmic sellers won the Buy Box {algorithmic_buybox_wins} times.")
print(f"Non-algorithmic sellers won the Buy Box {non_algorithmic_buybox_wins} times.")
